In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 📊 Production Planning Optimization using Linear Programming\n",
    "\n",
    "## Business Problem\n",
    "A furniture manufacturing company needs to determine the optimal production quantities for **Chairs**, **Tables**, and **Desks** to maximize profit while respecting resource constraints.\n",
    "\n",
    "### Key Objectives:\n",
    "- Maximize total profit\n",
    "- Optimize resource allocation\n",
    "- Meet minimum production requirements\n",
    "- Respect capacity constraints\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup and Imports"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import pulp\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from pathlib import Path\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Set visualization style\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (14, 8)\n",
    "plt.rcParams['font.size'] = 11\n",
    "\n",
    "print('✓ All libraries imported successfully!')\n",
    "print(f'PuLP version: {pulp.__version__}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Problem Definition\n",
    "\n",
    "### Decision Variables:\n",
    "- `x₁` = Number of Chairs to produce\n",
    "- `x₂` = Number of Tables to produce  \n",
    "- `x₃` = Number of Desks to produce\n",
    "\n",
    "### Objective Function:\n",
    "**Maximize Profit:** `Z = 45x₁ + 80x₂ + 120x₃`\n",
    "\n",
    "### Constraints:\n",
    "1. **Wood:** `5x₁ + 20x₂ + 30x₃ ≤ 5000` board feet\n",
    "2. **Metal:** `2x₁ + 4x₂ + 5x₃ ≤ 1500` kg\n",
    "3. **Fabric:** `3x₁ + 2x₂ + 0x₃ ≤ 800` yards\n",
    "4. **Labor:** `3x₁ + 5x₂ + 8x₃ ≤ 2000` hours\n",
    "5. **Machine Time:** `2x₁ + 3x₂ + 4x₃ ≤ 1200` hours\n",
    "6. **Storage:** `x₁ + x₂ + x₃ ≤ 500` units\n",
    "7. **Minimum Production:** `x₁ ≥ 50`, `x₂ ≥ 30`, `x₃ ≥ 20`\n",
    "8. **Non-negativity:** `x₁, x₂, x₃ ≥ 0`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define problem parameters\n",
    "products = ['Chairs', 'Tables', 'Desks']\n",
    "\n",
    "# Profit per unit ($)\n",
    "profit = {\n",
    "    'Chairs': 45,\n",
    "    'Tables': 80,\n",
    "    'Desks': 120\n",
    "}\n",
    "\n",
    "# Resource requirements\n",
    "resource_requirements = pd.DataFrame({\n",
    "    'Product': products,\n",
    "    'Wood (board ft)': [5, 20, 30],\n",
    "    'Metal (kg)': [2, 4, 5],\n",
    "    'Fabric (yards)': [3, 2, 0],\n",
    "    'Labor (hours)': [3, 5, 8],\n",
    "    'Machine (hours)': [2, 3, 4],\n",
    "    'Profit ($)': [45, 80, 120]\n",
    "})\n",
    "\n",
    "# Resource capacities\n",
    "capacity = {\n",
    "    'Wood': 5000,\n",
    "    'Metal': 1500,\n",
    "    'Fabric': 800,\n",
    "    'Labor': 2000,\n",
    "    'Machine': 1200,\n",
    "    'Storage': 500\n",
    "}\n",
    "\n",
    "# Minimum production requirements\n",
    "min_production = {\n",
    "    'Chairs': 50,\n",
    "    'Tables': 30,\n",
    "    'Desks': 20\n",
    "}\n",
    "\n",
    "# Display data\n",
    "print('Resource Requirements per Product:')\n",
    "display(resource_requirements)\n",
    "\n",
    "print('\\nResource Capacities:')\n",
    "for resource, cap in capacity.items():\n",
    "    print(f'  {resource}: {cap}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Build the Optimization Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create the LP model\n",
    "model = pulp.LpProblem('Furniture_Production_Optimization', pulp.LpMaximize)\n",
    "\n",
    "# Decision variables\n",
    "x = {\n",
    "    'Chairs': pulp.LpVariable('Chairs', lowBound=min_production['Chairs'], cat='Integer'),\n",
    "    'Tables': pulp.LpVariable('Tables', lowBound=min_production['Tables'], cat='Integer'),\n",
    "    'Desks': pulp.LpVariable('Desks', lowBound=min_production['Desks'], cat='Integer')\n",
    "}\n",
    "\n",
    "print('✓ Decision variables created:')\n",
    "for product, var in x.items():\n",
    "    print(f'  {product}: {var}')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Objective function: Maximize profit\n",
    "model += (\n",
    "    profit['Chairs'] * x['Chairs'] +\n",
    "    profit['Tables'] * x['Tables'] +\n",
    "    profit['Desks'] * x['Desks'],\n",
    "    'Total_Profit'\n",
    ")\n",
    "\n",
    "print('✓ Objective function added:')\n",
    "print(f'  Maximize: {model.objective}')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Add constraints\n",
    "\n",
    "# 1. Wood constraint\n",
    "model += (\n",
    "    5 * x['Chairs'] + 20 * x['Tables'] + 30 * x['Desks'] <= capacity['Wood'],\n",
    "    'Wood_Constraint'\n",
    ")\n",
    "\n",
    "# 2. Metal constraint\n",
    "model += (\n",
    "    2 * x['Chairs'] + 4 * x['Tables'] + 5 * x['Desks'] <= capacity['Metal'],\n",
    "    'Metal_Constraint'\n",
    ")\n",
    "\n",
    "# 3. Fabric constraint\n",
    "model += (\n",
    "    3 * x['Chairs'] + 2 * x['Tables'] + 0 * x['Desks'] <= capacity['Fabric'],\n",
    "    'Fabric_Constraint'\n",
    ")\n",
    "\n",
    "# 4. Labor constraint\n",
    "model += (\n",
    "    3 * x['Chairs'] + 5 * x['Tables'] + 8 * x['Desks'] <= capacity['Labor'],\n",
    "    'Labor_Constraint'\n",
    ")\n",
    "\n",
    "# 5. Machine time constraint\n",
    "model += (\n",
    "    2 * x['Chairs'] + 3 * x['Tables'] + 4 * x['Desks'] <= capacity['Machine'],\n",
    "    'Machine_Constraint'\n",
    ")\n",
    "\n",
    "# 6. Storage capacity constraint\n",
    "model += (\n",
    "    x['Chairs'] + x['Tables'] + x['Desks'] <= capacity['Storage'],\n",
    "    'Storage_Constraint'\n",
    ")\n",
    "\n",
    "print(f'✓ {len(model.constraints)} constraints added to the model')\n",
    "print('\\nConstraints:')\n",
    "for name, constraint in model.constraints.items():\n",
    "    print(f'  {name}: {constraint}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Solve the Optimization Problem"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Solve the model\n",
    "model.solve(pulp.PULP_CBC_CMD(msg=1))\n",
    "\n",
    "# Check status\n",
    "status = pulp.LpStatus[model.status]\n",
    "print(f'\\n{'='*60}')\n",
    "print(f'Solution Status: {status}')\n",
    "print(f'{'='*60}')\n",
    "\n",
    "if status == 'Optimal':\n",
    "    print('\\n✓ Optimal solution found!')\n",
    "else:\n",
    "    print('\\n✗ No optimal solution found!')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Extract and Display Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Extract optimal solution\n",
    "if model.status == pulp.LpStatusOptimal:\n",
    "    results = {\n",
    "        'Chairs': x['Chairs'].varValue,\n",
    "        'Tables': x['Tables'].varValue,\n",
    "        'Desks': x['Desks'].varValue\n",
    "    }\n",
    "    \n",
    "    total_profit = pulp.value(model.objective)\n",
    "    total_units = sum(results.values())\n",
    "    \n",
    "    print('\\n' + '='*60)\n",
    "    print('OPTIMAL PRODUCTION PLAN')\n",
    "    print('='*60)\n",
    "    \n",
    "    print('\\nProduction Quantities:')\n",
    "    for product, quantity in results.items():\n",
    "        revenue = quantity * profit[product]\n",
    "        print(f'  {product:10s}: {quantity:6.0f} units  (Revenue: ${revenue:10,.2f})')\n",
    "    \n",
    "    print(f'\\n  {'Total Units':10s}: {total_units:6.0f} units')\n",
    "    print(f'\\n{'─'*60}')\n",
    "    print(f'Maximum Total Profit: ${total_profit:,.2f}')\n",
    "    print('='*60)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create results DataFrame\n",
    "results_df = pd.DataFrame({\n",
    "    'Product': ['Chairs', 'Tables', 'Desks'],\n",
    "    'Quantity': [results['Chairs'], results['Tables'], results['Desks']],\n",
    "    'Unit Profit ($)': [profit['Chairs'], profit['Tables'], profit['Desks']],\n",
    "    'Total Revenue ($)': [\n",
    "        results['Chairs'] * profit['Chairs'],\n",
    "        results['Tables'] * profit['Tables'],\n",
    "        results['Desks'] * profit['Desks']\n",
    "    ]\n",
    "})\n",
    "\n",
    "# Add totals row\n",
    "totals = pd.DataFrame({\n",
    "    'Product': ['TOTAL'],\n",
    "    'Quantity': [results_df['Quantity'].sum()],\n",
    "    'Unit Profit ($)': ['—'],\n",
    "    'Total Revenue ($)': [results_df['Total Revenue ($)'].sum()]\n",
    "})\n",
    "\n",
    "results_df = pd.concat([results_df, totals], ignore_index=True)\n",
    "\n",
    "print('\\nDetailed Results Table:')\n",
    "display(results_df.style.format({\n",
    "    'Quantity': '{:.0f}',\n",
    "    'Total Revenue ($)': '${:,.2f}'\n",
    "}).set_properties(**{'text-align': 'center'}))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Resource Utilization Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate resource utilization\n",
    "chairs_qty = results['Chairs']\n",
    "tables_qty = results['Tables']\n",
    "desks_qty = results['Desks']\n",
    "\n",
    "resource_usage = {\n",
    "    'Wood': 5*chairs_qty + 20*tables_qty + 30*desks_qty,\n",
    "    'Metal': 2*chairs_qty + 4*tables_qty + 5*desks_qty,\n",
    "    'Fabric': 3*chairs_qty + 2*tables_qty + 0*desks_qty,\n",
    "    'Labor': 3*chairs_qty + 5*tables_qty + 8*desks_qty,\n",
    "    'Machine': 2*chairs_qty + 3*tables_qty + 4*desks_qty,\n",
    "    'Storage': chairs_qty + tables_qty + desks_qty\n",
    "}\n",
    "\n",
    "# Create utilization DataFrame\n",
    "utilization_data = []\n",
    "for resource, used in resource_usage.items():\n",
    "    cap = capacity[resource]\n",
    "    util_pct = (used / cap) * 100\n",
    "    slack = cap - used\n",
    "    \n",
    "    utilization_data.append({\n",
    "        'Resource': resource,\n",
    "        'Used': used,\n",
    "        'Capacity': cap,\n",
    "        'Utilization (%)': util_pct,\n",
    "        'Slack': slack,\n",
    "        'Status': 'Critical' if util_pct > 90 else 'High' if util_pct > 70 else 'Normal'\n",
    "    })\n",
    "\n",
    "utilization_df = pd.DataFrame(utilization_data)\n",
    "\n",
    "print('\\n' + '='*60)\n",
    "print('RESOURCE UTILIZATION ANALYSIS')\n",
    "print('='*60)\n",
    "display(utilization_df.style.format({\n",
    "    'Used': '{:.1f}',\n",
    "    'Capacity': '{:.0f}',\n",
    "    'Utilization (%)': '{:.2f}%',\n",
    "    'Slack': '{:.1f}'\n",
    "}).background_gradient(subset=['Utilization (%)'], cmap='RdYlGn_r'))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Sensitivity Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Shadow prices (dual values)\n",
    "print('\\n' + '='*60)\n",
    "print('SENSITIVITY ANALYSIS - SHADOW PRICES')\n",
    "print('='*60)\n",
    "print('\\nShadow prices indicate the marginal value of each constraint.')\n",
    "print('They show how much profit would increase if we had one more unit of that resource.\\n')\n",
    "\n",
    "shadow_prices = []\n",
    "for name, constraint in model.constraints.items():\n",
    "    shadow_price = constraint.pi\n",
    "    if shadow_price is not None:\n",
    "        shadow_prices.append({\n",
    "            'Constraint': name.replace('_', ' '),\n",
    "            'Shadow Price ($)': shadow_price,\n",
    "            'Interpretation': f'Adding 1 unit changes profit by ${shadow_price:.2f}'\n",
    "        })\n",
    "\n",
    "shadow_df = pd.DataFrame(shadow_prices)\n",
    "shadow_df = shadow_df.sort_values('Shadow Price ($)', ascending=False)\n",
    "\n",
    "display(shadow_df.style.format({'Shadow Price ($)': '${:.2f}'}))\n",
    "\n",
    "print('\\n💡 Key Insight:')\n",
    "binding = shadow_df[shadow_df['Shadow Price ($)'] > 0]\n",
    "if not binding.empty:\n",
    "    top_constraint = binding.iloc[0]\n",
    "    print(f\"   The most valuable resource is {top_constraint['Constraint']}\")\n",
    "    print(f\"   Shadow Price: ${top_constraint['Shadow Price ($)']:.2f} per unit\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Price sensitivity analysis\n",
    "print('\\n' + '='*60)\n",
    "print('PRICE SENSITIVITY ANALYSIS')\n",
    "print('='*60)\n",
    "print('\\nHow does profit change with price variations?\\n')\n",
    "\n",
    "price_changes = range(-20, 25, 5)\n",
    "sensitivity_results = []\n",
    "\n",
    "for product in products:\n",
    "    for change in price_changes:\n",
    "        new_price = profit[product] + change\n",
    "        profit_change = change * results[product]\n",
    "        new_total_profit = total_profit + profit_change\n",
    "        \n",
    "        sensitivity_results.append({\n",
    "            'Product': product,\n",
    "            'Price Change ($)': change,\n",
    "            'New Price ($)': new_price,\n",
    "            'New Total Profit ($)': new_total_profit,\n",
    "            'Profit Change ($)': profit_change\n",
    "        })\n",
    "\n",
    "sensitivity_df = pd.DataFrame(sensitivity_results)\n",
    "\n",
    "# Display sample\n",
    "print('Sample of sensitivity analysis (first 15 rows):')\n",
    "display(sensitivity_df.head(15).style.format({\n",
    "    'New Price ($)': '${:.0f}',\n",
    "    'New Total Profit ($)': '${:,.2f}',\n",
    "    'Profit Change ($)': '${:,.2f}'\n",
    "}))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Visualizations"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualization 1: Production Quantities\n",
    "fig, axes = plt.subplots(2, 2, figsize=(16, 12))\n",
    "colors = ['#3498db', '#2ecc71', '#e74c3c']\n",
    "\n",
    "# Plot 1: Production quantities\n",
    "quantities = [results[p] for p in products]\n",
    "bars = axes[0, 0].bar(products, quantities, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)\n",
    "axes[0, 0].set_title('Optimal Production Quantities', fontsize=16, fontweight='bold', pad=20)\n",
    "axes[0, 0].set_ylabel('Units Produced', fontsize=12)\n",
    "axes[0, 0].set_ylim(0, max(quantities) * 1.2)\n",
    "axes[0, 0].grid(axis='y', alpha=0.3, linestyle='--')\n",
    "\n",
    "# Add value labels\n",
    "for bar in bars:\n",
    "    height = bar.get_height()\n",
    "    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height,\n",
    "                    f'{int(height)}',\n",
    "                    ha='center', va='bottom', fontsize=12, fontweight='bold')\n",
    "\n",
    "# Plot 2: Resource utilization\n",
    "util_sorted = utilization_df.sort_values('Utilization (%)')\n",
    "colors_util = ['red' if x > 90 else 'orange' if x > 70 else 'green' \n",
    "               for x in util_sorted['Utilization (%)']]\n",
    "axes[0, 1].barh(util_sorted['Resource'], util_sorted['Utilization (%)'], \n",
    "                color=colors_util, alpha=0.7, edgecolor='black')\n",
    "axes[0, 1].set_xlabel('Utilization (%)', fontsize=12)\n",
    "axes[0, 1].set_title('Resource Utilization', fontsize=16, fontweight='bold', pad=20)\n",
    "axes[0, 1].axvline(x=90, color='orange', linestyle='--', linewidth=2, label='High Utilization (90%)')\n",
    "axes[0, 1].axvline(x=100, color='red', linestyle='--', linewidth=2, label='Full Capacity')\n",
    "axes[0, 1].legend(loc='lower right')\n",
    "axes[0, 1].grid(axis='x', alpha=0.3, linestyle='--')\n",
    "\n",
    "# Plot 3: Profit contribution\n",
    "profit_contribution = [results[p] * profit[p] for p in products]\n",
    "explode = (0.05, 0.05, 0.05)\n",
    "wedges, texts, autotexts = axes[1, 0].pie(profit_contribution, labels=products,\n",
    "                                            autopct='%1.1f%%', colors=colors,\n",
    "                                            startangle=90, explode=explode,\n",
    "                                            shadow=True)\n",
    "for autotext in autotexts:\n",
    "    autotext.set_color('white')\n",
    "    autotext.set_fontsize(12)\n",
    "    autotext.set_fontweight('bold')\n",
    "axes[1, 0].set_title('Profit Contribution by Product', fontsize=16, fontweight='bold', pad=20)\n",
    "\n",
    "# Plot 4: Resource slack\n",
    "slack_sorted = utilization_df.sort_values('Slack', ascending=True)\n",
    "axes[1, 1].barh(slack_sorted['Resource'], slack_sorted['Slack'], \n",
    "                color='#1abc9c', alpha=0.7, edgecolor='black')\n",
    "axes[1, 1].set_xlabel('Slack (Unused Capacity)', fontsize=12)\n",
    "axes[1, 1].set_title('Resource Slack Analysis', fontsize=16, fontweight='bold', pad=20)\n",
    "axes[1, 1].grid(axis='x', alpha=0.3, linestyle='--')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('../visualizations/optimization_results.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print('✓ Visualization saved: ../visualizations/optimization_results.png')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualization 2: Sensitivity Analysis\n",
    "fig, axes = plt.subplots(1, 2, figsize=(16, 6))\n",
    "\n",
    "# Plot 1: Price sensitivity\n",
    "for product in products:\n",
    "    product_data = sensitivity_df[sensitivity_df['Product'] == product]\n",
    "    axes[0].plot(product_data['New Price ($)'], \n",
    "                product_data['New Total Profit ($)'],\n",
    "                marker='o', label=product, linewidth=2.5, markersize=8)\n",
    "\n",
    "axes[0].set_xlabel('Product Price ($)', fontsize=12)\n",
    "axes[0].set_ylabel('Total Profit ($)', fontsize=12)\n",
    "axes[0].set_title('Profit Sensitivity to Price Changes', fontsize=16, fontweight='bold', pad=20)\n",
    "axes[0].legend(fontsize=11, loc='best')\n",
    "axes[0].grid(alpha=0.3, linestyle='--')\n",
    "axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))\n",
    "\n",
    "# Plot 2: Resource constraint tightness\n",
    "resource_names = utilization_df['Resource']\n",
    "utilization_pcts = utilization_df['Utilization (%)']\n",
    "colors_map = ['red' if u > 90 else 'orange' if u > 70 else 'green' for u in utilization_pcts]\n",
    "\n",
    "bars = axes[1].barh(resource_names, utilization_pcts, color=colors_map, alpha=0.7, edgecolor='black')\n",
    "axes[1].axvline(x=90, color='orange', linestyle='--', linewidth=2, label='High Utilization (90%)')\n",
    "axes[1].axvline(x=100, color='red', linestyle='--', linewidth=2, label='Full Capacity')\n",
    "axes[1].set_xlabel('Utilization (%)', fontsize=12)\n",
    "axes[1].set_title('Resource Constraint Tightness', fontsize=16, fontweight='bold', pad=20)\n",
    "axes[1].legend(fontsize=11, loc='lower right')\n",
    "axes[1].grid(axis='x', alpha=0.3, linestyle='--')\n",
    "\n",
    "# Add percentage labels\n",
    "for i, (bar, util) in enumerate(zip(bars, utilization_pcts)):\n",
    "    axes[1].text(util + 2, bar.get_y() + bar.get_height()/2, \n",
    "                f'{util:.1f}%', va='center', fontsize=10, fontweight='bold')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('../visualizations/sensitivity_analysis.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print('✓ Visualization saved: ../visualizations/sensitivity_analysis.png')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Export Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_